In [1]:
from bs4 import BeautifulSoup
import requests, re, sys
import pandas as pd
import os
import shutil

In [2]:
airports = pd.read_csv("in-airports.csv")

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
0,26555,VIDP,large_airport,Indira Gandhi International Airport,28.555630,77.095190,777.0,AS,India,IN,...,New Delhi,1,VIDP,DEL,NaN,http://www.newdelhiairport.in/,https://en.wikipedia.org/wiki/Indira_Gandhi_In...,Palam Air Force Station,51475,2022-02-11T17:31:57+00:00
1,26434,VABB,large_airport,Chhatrapati Shivaji International Airport,19.088699,72.867897,39.0,AS,India,IN,...,Mumbai,1,VABB,BOM,NaN,http://www.csia.in/,https://en.wikipedia.org/wiki/Chhatrapati_Shiv...,"Bombay, Sahar International Airport",1014475,2013-04-12T01:27:48+00:00
2,26618,VOMM,large_airport,Chennai International Airport,12.990005,80.169296,52.0,AS,India,IN,...,Chennai,1,VOMM,MAA,NaN,NaN,https://en.wikipedia.org/wiki/Chennai_Internat...,Madras,51150,2022-09-09T12:28:58+00:00
3,35145,VOBL,large_airport,Kempegowda International Airport,13.197900,77.706299,3000.0,AS,India,IN,...,Bangalore,1,VOBL,BLR,NaN,http://www.bengaluruairport.com/home/home.jspx,https://en.wikipedia.org/wiki/Kempegowda_Inter...,NaN,51200,2016-02-01T17:54:36+00:00
4,26444,VAGO,large_airport,Dabolim Airport,15.380800,73.831398,150.0,AS,India,IN,...,Vasco da Gama,1,VOGO,GOI,NaN,NaN,https://en.wikipedia.org/wiki/Dabolim_Airport,"Goa Airport, Dabolim Navy Airbase, दाबोळी विमानतळ",1000,2022-10-13T15:12:26+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,340017,IN-0118,heliport,Rushikonda Beach Heliport,17.779660,83.385490,NaN,AS,India,IN,...,Visakhapatnam,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2021-01-21T17:52:11+00:00
602,340018,IN-0119,heliport,Oxygen Towers Helipad,17.739990,83.314140,NaN,AS,India,IN,...,Visakhapatnam,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2021-01-21T17:52:56+00:00
603,341820,IN-0233,heliport,MIRC Heliport,19.050240,74.767188,NaN,AS,India,IN,...,Wakodi,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2021-03-08T16:56:16+00:00
604,340124,IN-0123,heliport,Yanam Heliport,16.726100,82.206740,NaN,AS,India,IN,...,Yanam,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2021-01-25T19:07:51+00:00


In [ ]:
loc1 = sys.argv[1]
loc2 = sys.argv[2]

In [7]:
main_url = "https://uk.flightaware.com"
url_extract = requests.get(main_url + "/live/findflight?origin={}&destination={}".format(loc1, loc2)).text
soup = BeautifulSoup(url_extract, 'lxml')

<!DOCTYPE html>
<html class="responsive-full" dir="LTR" lang="en" prefix="og: http://ogp.me/ns#">
<!--
Looking at the source?  Great!
  FlightAware is looking for UI/UX, web, and backend developers, mathematicians and electrical engineers.

Get in touch with us:
https://flightaware.com/about/careers

-->
<head>
<!-- 	Attention! You must agree with FlightAware's Terms of Use at http://flightaware.com/about/termsofuse
  Data robots and collection agents should use http://flightaware.com/commercial/flightxml/
            Copyright 2023 FlightAware.  All Rights Reserved. -->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="@FlightAware" name="twitter:site"/>
<meta content="ijk4kalzz9je9chk38y60825grrr8m" name="facebook-domain-verification"/>
<title>Flight Finder ✈ Rajiv Gandhi Int'l (VOHS) - Chatrapati Shivaji Int'l (VABB)  - FlightAware</title>
<meta content="width=device-width, initial-scale=1, user

In [8]:
table = soup.find_all('tr', class_ = "ffinder-results-row-bordertop ffinder-results-row")

In [9]:
set1 = set()
for i in table:
    x = re.findall(r'url="/live/flight/id/[A-Z0-9]+', str(i))[0][21:]
    set1.add(x)

{'AIC616',
 'AIC618',
 'AIC698',
 'BDA154',
 'BDA654',
 'IGO163',
 'IGO2441',
 'IGO5245',
 'IGO5257',
 'IGO5268',
 'IGO5384',
 'IGO5389',
 'IGO6189',
 'IGO6591',
 'IGO773',
 'N600LP',
 'T7SKE',
 'VTI872',
 'VTI874',
 'VTI876',
 'VTI878',
 'VTOMM',
 'VTZAP'}

In [ ]:
import random
import subprocess
try:
    while True:
        flight = random.choice(list(set1))
        url_extract = requests.get(main_url + "/live/flight/{}/history".format(flight)).text
        soup = BeautifulSoup(url_extract, 'lxml')
        new_table = soup.find('table', class_ = "prettyTable fullWidth tablesaw tablesaw-stack")
        table_body = new_table.find_all('tr')[1:]
        while True:
            if 'Scheduled' in table_body[0].text or 'On The Way!' in table_body[0].text:
                table_body.pop(0)
            else:
                break
        flight_links = []
        for i in table_body:
            if loc1 in i.text and loc2 in i.text and 'Cancelled' not in i.text:
                x = re.findall(r'a href="[/a-zA-Z0-9]+', str(i))[0][8:]
                flight_links.append(x)
                if len(flight_links) == 5:
                    break
        elevation1 = airports[airports['gps_code'] == loc1].reset_index(drop=True)['elevation_ft'][0]*0.3048
        elevation2 = airports[airports['gps_code'] == loc2].reset_index(drop=True)['elevation_ft'][0]*0.3048
        if len(flight_links) == 5:
            path = os.getcwd() + r"\Datasets"
            shutil.rmtree(path)
            os.mkdir(path)
            for i in range(5):
                print(subprocess.run("ipython scraping.ipynb {} {} {}".format(main_url+flight_links[i]+"/tracklog", elevation1, elevation2), capture_output=True).stdout.decode('UTF-8'))
            break
        else:
            set1.remove(flight)
except Exception:
    print("No flights are there between {} and {}, please change the locations and try again.".format(airports[airports['gps_code']==loc1].reset_index()['municipality'][0], airports[airports['gps_code']==loc2].reset_index()['municipality'][0]))
    

'VTI874'